# Предиктор результатов футбольных матчей

In [1]:
#importing
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

## Обзор данных

In [14]:
data = pd.read_csv("data.csv")
data.head()

,idmatch,odds_open_win1,odds_open_draw,odds_open_win2,vi_win_home,vi_win_away,vi_nolose_home,vi_nolose_away,vi_over25_home,vi_over25_away,...,team2_away_xgpower_03_draw_xga90index,team2_away_xgpower_03_draw_xgsubxgaindex,team2_away_xgpower_03_cop_xg90index,team2_away_xgpower_03_cop_xga90index,team2_away_xgpower_03_cop_xgsubxgaindex,team2_away_xgpower_03_com_xg90index,team2_away_xgpower_03_com_xga90index,team2_away_xgpower_03_com_xgsubxgaindex,result_finish_match,result_total_match
0,39231,2.53,3.15,3.02,2.255,-4.318,19.259,-3.855,-41.798,-10.410,...,0.98,0.40,2.13,4.88,0.44,0.55,2.79,0.20,2.0,0.0
1,39232,2.16,3.26,3.66,-13.230,-29.496,4.164,-47.122,-43.926,-4.662,...,1.23,0.36,0.00,0.00,0.00,0.51,1.34,0.38,1.0,0.0
2,39233,2.35,3.16,3.31,-35.270,-1.048,-17.721,-2.781,8.467,-6.808,...,0.87,0.31,0.00,0.17,0.00,0.00,0.00,0.00,3.0,1.0
3,39235,2.50,3.25,2.97,17.593,27.970,1.079,10.260,-27.452,-23.279,...,1.99,0.05,0.00,0.00,0.00,0.18,0.80,0.22,3.0,1.0
4,45425,2.15,3.49,3.46,19.656,-22.231,-8.999,-8.685,7.560,-7.321,...,0.64,1.45,3.35,1.40,2.40,0.73,2.48,0.29,1.0,1.0


In [47]:

with open("raw_data_info.txt", mode='w') as df_info:
    data.info(verbose=True, show_counts=True, buf=df_info)
print('\nБаланс классов\n')
print(data['result_finish_match'].value_counts(), '\n')
print(data['result_total_match'].value_counts())



Баланс классов

1.0    24763
3.0    17585
2.0    15099
Name: result_finish_match, dtype: int64 

0.0    29307
1.0    28140
Name: result_total_match, dtype: int64
